In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
import re

locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

'pt_BR.utf8'

In [ ]:
def str_cleasing(str):
    str = re.sub(r'\n', ' ', str)
    str = re.sub(r'\s+', ' ', str)
    str = re.sub(r'\s+$', ' ', str)
    str = re.sub(r'^\s+', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    return str

In [2]:
class ODia:

    def __init__(self, base_url='https://odia.ig.com.br/'):
        self.base_url = base_url
    
    def yesterday_url_str(self):
        ontem = datetime.now() - timedelta(days=1)
        data_formatada = ontem.strftime('%d/%m/%Y')
        return data_formatada
    

    def getUrls(self, url, word):
        try:
            browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
            page = requests.get(url, headers = browsers)
            resposta = page.text
            soup = BeautifulSoup(resposta, 'html.parser')
            allHrefs = soup.find_all('a')

            selectedUrls = []
            for a in allHrefs:
                newsUrl = a.get('href')
                if newsUrl is not None and word in newsUrl and not newsUrl.endswith(word):
                    selectedUrls.append(f"https://odia.ig.com.br{newsUrl}")
            selectedUrls = list(dict.fromkeys(selectedUrls))
            return selectedUrls
        except:
            return []

    def getNewsData(self, url, only_yesterday=True):
        try:
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
            page = requests.get(url, headers = headers)
            
            resposta = page.text
            soup = BeautifulSoup(resposta, 'html.parser')

            title = soup.find('h1').get_text()
            title = str_cleasing(title)
            title = re.sub(r'\s+', ' ', title).strip()
            
            timestamp = soup.find('time', class_="published").get_text()
            isYesterday = self.yesterday_url_str() in timestamp
            
            content = ""
            contentHtml = soup.find_all('div', class_="texto")
            for c in contentHtml:
                content += c.get_text()
                content = str_cleasing(content)
                content += '\n'
            
            if only_yesterday and not isYesterday:
                return None
        
            return {
                "url": url,
                "title": title,
                "content": content
            }
        except:
            return None
